# 隐马尔科夫模型

### 杜小勤，武汉纺织大学数学与计算机学院，2020年03月25日
https://github.com/duxiaoqin/Lecture-notes-for-Machine-Learning

HMM模型的实现

In [1]:
import numpy as np
import copy

In [2]:
class HiddenMarkovModel:
    #Q：状态集合，V：观测集合
    def __init__(self, Q, V, threshold = 1e-6):
        self.Q, self.V, self.threshold = Q, V, threshold

    #O：观测序列，A：状态转移概率，B：观测概率分布，pi：初始概率分布
    def forward(self, O, A, B, pi):
        N, T = len(self.Q), len(O)
        self.alphas = np.zeros((T, N)) #保存各阶段的alpha值
        self.alphas[0] = pi * B[:, self.V.index(O[0])] #初始化alpha[0]
        for t in range(1, T): #递推计算各阶段alpha值
            self.alphas[t] = np.dot(self.alphas[t-1], A) * B[:, self.V.index(O[t])]
        return np.sum(self.alphas[T-1]) #返回观测序列O的概率
    
    #O：观测序列，A：状态转移概率，B：观测概率分布，pi：初始概率分布
    def backward(self, O, A, B, pi):
        N, T = len(self.Q), len(O)
        self.betas = np.zeros((T, N)) #保存各阶段的beta值
        self.betas[T-1] = np.ones(N) #初始化beta[T-1]
        for t in range(T-2, -1, -1): #反向递推计算各阶段beta值
            self.betas[t] = np.dot(self.betas[t+1] * B[:, self.V.index(O[t+1])], A.T)
        return np.dot(pi * B[:, self.V.index(O[0])], self.betas[0]) #返回观测序列O的概率
    
    #O：观测序列，A：状态转移概率，B：观测概率分布，pi：初始概率分布
    def viterbi(self, O, A, B, pi):
        N, T = len(self.Q), len(O)
        self.deltas = np.zeros((T, N)) #保存各阶段的delta值
        self.psis = np.zeros((T, N), dtype = np.int16) #保存各阶段的psi值
        self.deltas[0] = pi * B[:, self.V.index(O[0])] #初始化delta[0]
        for t in range(1, T): #递推计算各阶段delta值
            last = self.deltas[t-1].reshape(-1, 1) * A
            self.deltas[t] = np.max(last, axis = 0) * B[:, self.V.index(O[t])]
            self.psis[t] = np.argmax(last, axis = 0)
        pstar = np.max(self.deltas[T-1])
        path = [np.argmax(self.deltas[T-1])]
        for t in range(T-1, 0, -1):
            path.insert(0, self.psis[t, path[0]])
        return pstar, [index+1 for index in path] #返回与观测序列O对应的最优概率与最优状态
    
    #使用Baum-Welch算法训练HMM模型
    #O：观测序列矩阵（多个样本），A、B、pi：初始概率
    def fit(self, O, A, B, pi, n_iter = 100):
        N, M, D, T = len(self.Q), len(self.V), len(O), len(O[0])
        self.A, self.B, self.pi = copy.deepcopy(A), copy.deepcopy(B), copy.deepcopy(pi)
        #迭代所需参数初始化
        gamma, xi, mask = np.zeros((D, T, N)), np.zeros((D, T, N, N)), np.zeros((D, T, N, M))
        for step in range(n_iter):
            #执行E-step
            for d, o in enumerate(O): #遍历每个观测序列
                p = self.forward(o, self.A, self.B, self.pi)
                p = self.backward(o, self.A, self.B, self.pi)
                gamma[d] = self.alphas * self.betas / p
                for t in range(T-1):
                    xi[d, t] = self.alphas[t].reshape(-1, 1) * self.A * self.B[:, self.V.index(o[t+1])].reshape(1, -1) * self.betas[t+1] / p
                    mask[d, t, :, self.V.index(o[t])] = np.ones(N)
                mask[d, T-1, :, self.V.index(o[T-1])] = np.ones(N)
            #执行M-step
            old_A, old_B, old_pi = copy.deepcopy(self.A), copy.deepcopy(self.B), copy.deepcopy(self.pi)
            self.A = np.sum(np.sum(xi[:, :-1, :, :], axis = 1), axis = 0) / np.sum(np.sum(gamma[:, :-1, :], axis = 1), axis = 0).reshape(-1, 1)
            self.B = np.sum(np.sum(gamma.reshape(D, T, N, 1)*mask, axis = 1), axis = 0) / np.sum(np.sum(gamma, axis = 1), axis = 0).reshape(-1, 1)
            self.pi = np.sum(gamma[:, 0, :], axis = 0) / D 
            if max(np.abs(self.A - old_A).max(), np.abs(self.B - old_B).max(), np.abs(self.pi - old_pi).max()) < self.threshold:
                break
        print(self.A)
        print(self.B)
        print(self.pi)

测试：使用前向算法与后向算法计算给定观测序列的概率；使用Viterbi算法求解最优概率与最优状态序列

验证例10.2与习题10.1

设置模型参数

In [3]:
Q = ['1', '2', '3'] #状态集合
V = ['红', '白'] #观测集合

#模型参数
A = [[0.5, 0.2, 0.3],
      [0.3, 0.5, 0.2],
      [0.2, 0.3, 0.5]]
B = [[0.5, 0.5],
      [0.4, 0.6],
      [0.7, 0.3]]
pi = [0.2, 0.4, 0.4]

测试自定义的HMM模型

In [4]:
myhmm = HiddenMarkovModel(Q, V)
O = ['红', '白', '红']
print(myhmm.forward(O, np.array(A), np.array(B), np.array(pi)))
print(myhmm.backward(O, np.array(A), np.array(B), np.array(pi)))
print(myhmm.viterbi(O, np.array(A), np.array(B), np.array(pi)))

0.130218
0.130218
(0.014699999999999998, [3, 3, 3])


使用hmmlearn库

In [5]:
#在anaconda3下安装：pip install hmmlearn
from hmmlearn import hmm

In [6]:
model = hmm.MultinomialHMM(n_components = len(Q))
model.startprob_ = np.array(pi)
model.transmat_ = np.array(A)
model.emissionprob_ = np.array(B)
O = np.array([0, 1, 0]).reshape(-1, 1) #0：红，1：白
logprob = model.score(O) #观测序列的log概率
print(np.exp(logprob))
logprob, states = model.decode(O, algorithm = 'viterbi') #观测序列对应的最佳隐状态
print(np.exp(logprob), [state+1 for state in states])

0.130218
0.0147 [3, 3, 3]


In [7]:
#自定义HMM
O = ['红', '白', '红', '白']
print(myhmm.forward(O, np.array(A), np.array(B), np.array(pi)))
print(myhmm.backward(O, np.array(A), np.array(B), np.array(pi)))
print(myhmm.viterbi(O, np.array(A), np.array(B), np.array(pi)))
#库HMM
O = np.array([0, 1, 0, 1]).reshape(-1, 1) #0：红，1：白
print(np.exp(model.score(O)))
logprob, states = model.decode(O, algorithm = 'viterbi')
print(np.exp(logprob), [state+1 for state in states])

0.0600908
0.0600908
(0.0030239999999999993, [3, 2, 2, 2])
0.0600908
0.003024 [3, 2, 2, 2]


习题10.2

In [8]:
Q = ['1', '2', '3'] #状态集合
V = ['红', '白'] #观测集合

#模型参数
A = [[0.5, 0.1, 0.4], 
     [0.3, 0.5, 0.2], 
     [0.2, 0.2, 0.6]]
B = [[0.5, 0.5], 
     [0.4, 0.6], 
     [0.7, 0.3]]
pi = [0.2, 0.3, 0.5]

In [9]:
#自定义HMM
myhmm = HiddenMarkovModel(Q, V)
O = ['红', '白', '红', '红', '白', '红', '白', '白']
print(myhmm.forward(O, np.array(A), np.array(B), np.array(pi)))
print(myhmm.backward(O, np.array(A), np.array(B), np.array(pi)))
print(myhmm.viterbi(O, np.array(A), np.array(B), np.array(pi)))
#库HMM
model = hmm.MultinomialHMM(n_components = len(Q))
O = np.array([0, 1, 0, 0, 1, 0, 1, 1]).reshape(-1, 1) #0：红，1：白
model.startprob_ = np.array(pi)
model.transmat_ = np.array(A)
model.emissionprob_ = np.array(B)
print(np.exp(model.score(O)))
logprob, states = model.decode(O, algorithm = 'viterbi')
print(np.exp(logprob), [state+1 for state in states])

0.00347670944928
0.00347670944928
(3.0245685119999991e-05, [3, 3, 3, 3, 3, 3, 2, 2])
0.00347670944928
3.024568512e-05 [3, 3, 3, 3, 3, 3, 2, 2]


模型参数的学习：Baum-Welch算法的验证

从文件导入测试数据

In [10]:
import pandas as pd
data = pd.read_csv('data_python.csv')
 
O = data['Visible'].values

In [11]:
#状态与观测集合
Q = ['A', 'B'] #状态集合
V = [0, 1, 2] #观测集合
N, M = len(Q), len(V)
#初始化A、B、pi：使用固定参数
A = np.ones((N, N))
A = A / np.sum(A, axis = 1).reshape(-1, 1)
B = np.array(((1, 2, 3), (4, 5, 6)))
B = B / np.sum(B, axis = 1).reshape(-1, 1)
pi = np.array((0.5, 0.5))
#自定义HMM
myhmm = HiddenMarkovModel(Q, V)
myhmm.fit([O], A, B, pi)
print('*'*40)
#库HMM
model = hmm.MultinomialHMM(n_components = N, n_iter = 100, tol = 1e-6)
model.startprob = pi
model.transmat = A
model.emissionprob_ = B
model.init_params = 'st'
model.fit(O.reshape(-1, 1))
print(model.transmat_)
print(model.emissionprob_)
print(model.startprob_)

[[ 0.77384134  0.22615866]
 [ 0.2278316   0.7721684 ]]
[[ 0.07801696  0.17193173  0.7500513 ]
 [ 0.33355442  0.36773984  0.29870574]]
[  9.39942699e-51   1.00000000e+00]
****************************************
[[ 0.77384134  0.22615866]
 [ 0.2278316   0.7721684 ]]
[[ 0.07801696  0.17193173  0.7500513 ]
 [ 0.33355442  0.36773984  0.29870574]]
[  9.39942699e-51   1.00000000e+00]


In [12]:
#初始化A、B、pi：B和pi使用随机参数
A = np.ones((N, N))
A = A / np.sum(A, axis = 1).reshape(-1, 1)
B = np.random.rand(N, M)
B = B / np.sum(B, axis = 1).reshape(-1, 1)
pi = np.random.rand(N)
pi = pi / np.sum(pi)
#自定义HMM
myhmm = HiddenMarkovModel(Q, V)
myhmm.fit([O], A, B, pi)
print('*'*40)
#库HMM
model = hmm.MultinomialHMM(n_components = N, n_iter = 100, tol = 1e-6)
model.startprob = pi
model.transmat = A
model.emissionprob_ = B
model.init_params = 'st'
model.fit(O.reshape(-1, 1))
print(model.transmat_)
print(model.emissionprob_)
print(model.startprob_)

[[ 0.60933483  0.39066517]
 [ 0.18197208  0.81802792]]
[[ 0.41127129  0.41308927  0.17563944]
 [ 0.10919346  0.20251869  0.68828785]]
[  1.00000000e+00   8.09753825e-79]
****************************************
[[ 0.6092341   0.3907659 ]
 [ 0.18199494  0.81800506]]
[[ 0.41132036  0.41310339  0.17557625]
 [ 0.10918331  0.20252108  0.68829562]]
[  1.00000000e+00   7.38479742e-79]


In [13]:
#初始化A、B、pi：全部使用随机参数
A = np.random.rand(N, N)
A = A / np.sum(A, axis = 1).reshape(-1, 1)
B = np.random.rand(N, M)
B = B / np.sum(B, axis = 1).reshape(-1, 1)
pi = np.random.rand(N)
pi = pi / np.sum(pi)
#自定义HMM
myhmm = HiddenMarkovModel(Q, V)
myhmm.fit([O], A, B, pi)
print('*'*40)
#库HMM
model = hmm.MultinomialHMM(n_components = N, n_iter = 100, tol = 1e-6)
model.startprob = pi
model.transmat = A
model.emissionprob_ = B
model.init_params = 'st'
model.fit(O.reshape(-1, 1))
print(model.transmat_)
print(model.emissionprob_)
print(model.startprob_)

[[ 0.90391605  0.09608395]
 [ 0.13815637  0.86184363]]
[[ 0.2884067   0.34442701  0.36716629]
 [ 0.08546423  0.16113607  0.75339969]]
[  1.00000000e+00   8.58605653e-67]
****************************************
[[ 0.67514367  0.32485633]
 [ 0.33398649  0.66601351]]
[[ 0.38345488  0.35659649  0.25994862]
 [ 0.02227811  0.18034526  0.79737663]]
[  1.00000000e+000   8.08048669e-140]


说明：Baum-Welch算法的训练结果与提供的样本数量、算法的迭代次数与阈值、算法的终止策略等有很大的关系。

## 参考文献：
- https://github.com/wzyonggege/statistical-learning-method
- 李航。《统计学习方法》，清华大学出版社，2012年3月第1版；
- http://www.cnblogs.com/pinard/p/6972299.html
- https://www.cnblogs.com/pinard/p/7001397.html
- https://xbuba.com/questions/38628872
- http://www.adeveloperdiary.com/data-science/machine-learning/derivation-and-implementation-of-baum-welch-algorithm-for-hidden-markov-model/